# Convolutional Neural Networks

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
#convert MNIST image files into a Tensor of 4-dimensions(# of images, height, width, color channels)
transform = transforms.ToTensor()

In [6]:
train_data = datasets.MNIST(root = '/cnn_data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root = '/cnn_data', train=False, download=True, transform=transform)

In [7]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [8]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [9]:
pwd

'd:\\GitHub\\Deep-Learning-with-Pytorch'

In [10]:
ls

 Volume in drive D is New Volume
 Volume Serial Number is 587D-D239

 Directory of d:\GitHub\Deep-Learning-with-Pytorch

10/19/2024  12:15 AM    <DIR>          .
10/20/2024  08:22 PM    <DIR>          ..
10/11/2024  01:27 AM             3,301 .gitignore
10/15/2024  01:18 PM             3,494 iris_model.pt
10/11/2024  01:23 AM            41,116 Lesson1.ipynb
10/20/2024  11:41 PM            67,264 Lesson2.ipynb
10/19/2024  01:55 PM             7,802 Lesson3.ipynb
10/11/2024  01:27 AM                28 README.md
               6 File(s)        123,005 bytes
               2 Dir(s)  153,607,925,760 bytes free


In [11]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [ ]:
conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [10]:
# Grab one image in MNIST train data

for i, (X_train, y_train) in enumerate(train_data):
    break

In [22]:
X_train.shape

torch.Size([1, 28, 28])

In [23]:
x = X_train.view(1, 1 , 28, 28)
x.shape

torch.Size([1, 1, 28, 28])

In [24]:
x = X_train.view(1, 1 , 28, 28)
x = F.relu(conv1(x))
x.shape

torch.Size([1, 6, 26, 26])

1 single image, 6 filters and 26x26 image

In [25]:
x = F.max_pool2d(x, kernel_size=2, stride=2)
x.shape

torch.Size([1, 6, 13, 13])

In [26]:
x = F.relu(conv2(x))
x.shape

torch.Size([1, 16, 11, 11])

In [27]:
x = F.max_pool2d(x, kernel_size=2, stride=2)
x.shape

torch.Size([1, 16, 5, 5])

In [12]:
class CNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        
        #Fully connected layer
        self.fc1 = nn.Linear(5 * 5 *16, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, X):
        # first pass
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        # second pass
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        
        # Re-view to flatten it out
        X = X.view(-1, 5 * 5 * 16)
        
        # Fully connected layers
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

In [13]:
model = CNN_model()
model

CNN_model(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [14]:
criterian = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
import time
start_time = time.time()

# create varibales to track things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    train_corr = 0
    test_corr = 0
    
    # Train
    for b, (X_train, y_train) in enumerate(train_loader):
        b += 1  #start batch number at 1
        y_pred = model(X_train)
        loss = criterian(y_pred, y_train)
        
        predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions. Indexed off the firdt point
        batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True=1, False=0, sum those up
        train_corr += batch_corr
        
        # update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print out some results
        if b%1000 == 0:
            print(f"Epoch: {i + 1} Batch: {b} Loss: {loss.item()}")
            
    train_losses.append(loss)
    train_correct.append(train_corr)
    
    #Test
    with torch.no_grad(): # No gradient so we don't update our weights and biases with test data
        for b, (X_test, y_test) in enumerate(test_loader):
            y_val = model(X_test)
            loss = criterian(y_pred, y_train)
        
            predicted = torch.max(y_val.data, 1)[1] # adding up the correct predictions
            tst_corr = (predicted == y_train).sum() # how many we got correct from this batch. True=1, False=0, sum those up
            
    test_losses.append(loss)
    test_correct.append(tst_corr)
    
current_time = time.time()
total_time = (current_time - start_time)
print(f"Time: {total_time/60} minutes")
            

Epoch: 0 Batch: 1000 Loss: 0.08038105070590973
Epoch: 0 Batch: 2000 Loss: 0.8423039317131042
Epoch: 0 Batch: 3000 Loss: 0.045939452946186066
Epoch: 0 Batch: 4000 Loss: 0.2777944803237915
Epoch: 0 Batch: 5000 Loss: 0.0052370186895132065
Epoch: 0 Batch: 6000 Loss: 0.1720704436302185
Epoch: 1 Batch: 1000 Loss: 0.0203254371881485
Epoch: 1 Batch: 2000 Loss: 0.2521364092826843
Epoch: 1 Batch: 3000 Loss: 0.0033085779286921024
Epoch: 1 Batch: 4000 Loss: 0.11261129379272461
Epoch: 1 Batch: 5000 Loss: 0.0016869606915861368
Epoch: 1 Batch: 6000 Loss: 0.002958751982077956
Epoch: 2 Batch: 1000 Loss: 0.15150240063667297
Epoch: 2 Batch: 2000 Loss: 0.10426217317581177
Epoch: 2 Batch: 3000 Loss: 0.004305713810026646
Epoch: 2 Batch: 4000 Loss: 0.42686066031455994
Epoch: 2 Batch: 5000 Loss: 0.0028802831657230854
Epoch: 2 Batch: 6000 Loss: 0.005850939080119133
Epoch: 3 Batch: 1000 Loss: 0.07497389614582062
Epoch: 3 Batch: 2000 Loss: 0.001166999340057373
Epoch: 3 Batch: 3000 Loss: 6.134933937573805e-05
Epo